In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os, io, sys, subprocess
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr

In [2]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')

In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [ ]:
system_message = "You are an AI assitant that assists in converting/reimplementing Python code to C++ code for a windows machine. "
system_message += "Respond only with C++ code with occasional comments and do not provide any explanations. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [6]:
def user_prompt_for(python):
    user_prompt = "Rewrite the following Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code with occasional comments and do not provide any explanations. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [7]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [8]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp","w") as f:
        f.write(code)

In [9]:
def optimize_gpt(python):
    stream = openai.chat.completions.create(
        model= OPENAI_MODEL,
        messages= messages_for(python),
        stream= True
    )
    reply = ""
    for chunck in stream:
        fragment = chunck.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end = '', flush=True)
    write_output(reply)

In [14]:
def stream_gpt(python):
    stream = openai.chat.completions.create(
        model= OPENAI_MODEL,
        messages= messages_for(python),
        stream= True
    )
    reply = ""
    for chunck in stream:
        fragment = chunck.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [10]:
def optimize_claude(python):
    result = claude.messages.stream(
        model= CLAUDE_MODEL,
        max_tokens= 2000,
        system = system_message,
        messages= [{"role": "user", "content": user_prompt_for(python)}]
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end = "",flush=True)
    write_output(reply)

In [15]:
def stream_claude(python):
    result = claude.messages.stream(
        model= CLAUDE_MODEL,
        max_tokens= 2000,
        system = system_message,
        messages= [{"role": "user", "content": user_prompt_for(python)}]
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [16]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far    

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
exec(pi)

Result: 3.141592658589
Execution Time: 14.230985 seconds


In [13]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

// Function calculates the result based on given iterations and parameters
double calculate(int iterations, double param1, double param2) {
    double result = 1.0;
    for (double i = 1; i <= iterations; ++i) {
        double j1 = i * param1 - param2;
        result -= (1/j1);
        double j2 = i * param1 + param2;
        result += (1/j2);
    }
    return result;
}

int main() {
    // Get the start time
    auto start_time = std::chrono::high_resolution_clock::now();

    // Calculate the result
    double result = calculate(100000000, 4.0, 1.0) * 4;

    // Get the end time
    auto end_time = std::chrono::high_resolution_clock::now();

    // Calculate the execution time
    std::chrono::duration<double> execution_time = end_time - start_time;

    // Print the result with 12 decimal places
    std::cout << "Result: " << std::fixed << std::setprecision(12) << result << std::endl;

    // Print the execution time i

In [17]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.
